In [105]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [106]:
def load_data_description(file="full_ap_description.csv"):
    return pd.read_csv(file)

def expand_coordinates(coordinate):
    if isinstance(coordinate, str):
        return coordinate.replace(")", "").replace("(", "").replace("'", "").split(",")
    else:
        return coordinate.split(",")

In [107]:
FOLDER = "/home/cozzolin/edge_cnn_scheduling/sim/data/wlan.lrz-data"
AP_DESCR = "/home/cozzolin/edge_cnn_scheduling/sim/data/full_ap_description.csv"
BUILDING = [
    "LMU, Geb. 0000M, Hauptgebäude/MitteltraktGeschwister-Scholl-Platz 180539 München",
    "LMU, Geb. 0030, Hauptgebäude inkl. Turmgebäude, BibliothekGeschwister-Scholl-Platz 180539 München",
    "LMU, Fachbibliothek PhilologicumLudwigstr. 2580539 München",
    "TUM, Geb. 5406, Chemiegebäude Bau Ch6Lichtenbergstrasse 485748 Garching",
    "HM, Gebäude GLothstr. 3480335 München"
    ]

In [108]:
print("Loading data from APs ...")
description = load_data_description(AP_DESCR)
# location = BUILDING
location = description.Standort.unique()
filtered_desc = description[(description['Standort'].isin(location))][["ap", "location", "Standort"]]

filtered_desc['latitude'] = filtered_desc.apply(lambda row: expand_coordinates(row.location)[0], axis = 1) 
filtered_desc['longitude'] = filtered_desc.apply(lambda row: expand_coordinates(row.location)[1], axis = 1) 
filtered_desc = filtered_desc.loc[:, ['ap', 'latitude', 'longitude', "Standort"]]    
filtered_desc = filtered_desc.reset_index(drop=True)

coord_ap_data = pd.DataFrame(columns=['timestamp', 'total', 'latitude', 'longitude', 'location'])
# TODO: Create new dataframe with timestamp and one column for each building?

Loading data from APs ...


In [109]:
coord_dataframe_list = []
prev = None

for idx, row in tqdm(filtered_desc.iterrows(), total=len(filtered_desc)):
    fd = row.ap
    try:
        ap_data = pd.read_csv('{}/{}.gz'.format(FOLDER, fd), compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False)

        # Get the total amount of users connected to the AP for all networks
        col_list = list(ap_data)
        col_list.remove('timestamp')
        ap_data.fillna(0, inplace=True)
        ap_data = ap_data.assign(total=np.ceil(ap_data[col_list].sum(axis=1)))

        # Filter the dataframe to output only timestamp and total
        ap_data = ap_data.loc[:, ['timestamp', 'total']]    
        ap_data.drop_duplicates(subset=['timestamp'], inplace=True)
        ap_data.set_index('timestamp', inplace=True)
        #ap_data['timestamp'] = ap_data['timestamp'] // 100
        #ap_data.columns = [str(col) if idx == 0 else fd + "." + str(col) for idx, col in enumerate(ap_data.columns)]
        ap_data.columns = [fd + "." + str(col) for col in ap_data.columns]
        data = ap_data if idx == 0 else pd.concat([data, ap_data], axis=1)

        if prev is None or row.Standort == prev:
            coord_data = ap_data if idx == 0 else pd.concat([coord_data, ap_data], axis=1)
        else:
    #         print("Detected location change")
            coord_data.dropna(inplace=True)
            coord_data = coord_data.assign(total=coord_data.sum(axis=1)).loc[:, ['total']]  
            coord_data = coord_data.assign(latitude=row.latitude)
            coord_data = coord_data.assign(longitude=row.longitude)
            coord_data = coord_data.assign(location=row.Standort)
            coord_dataframe_list.append(coord_data)
            coord_data = pd.DataFrame()

        prev = row.Standort
    except FileNotFoundError:
        pass
        
coord_data.dropna(inplace=True)
coord_data = coord_data.assign(total=coord_data.sum(axis=1)).loc[:, ['total']]  
coord_data = coord_data.assign(latitude=row.latitude)
coord_data = coord_data.assign(longitude=row.longitude)
coord_data = coord_data.assign(location=row.Standort)
coord_dataframe_list.append(coord_data)




  0%|          | 0/4418 [00:00<?, ?it/s]


  0%|          | 2/4418 [00:00<04:09, 17.67it/s]


  0%|          | 4/4418 [00:00<04:13, 17.38it/s]


  0%|          | 6/4418 [00:00<04:23, 16.73it/s]


  0%|          | 8/4418 [00:00<04:39, 15.79it/s]


  0%|          | 10/4418 [00:00<05:04, 14.45it/s]


  0%|          | 12/4418 [00:00<05:01, 14.60it/s]


  0%|          | 14/4418 [00:00<04:56, 14.86it/s]


  0%|          | 16/4418 [00:01<04:45, 15.39it/s]


  0%|          | 18/4418 [00:01<04:39, 15.76it/s]


  0%|          | 20/4418 [00:01<04:36, 15.93it/s]


  0%|          | 22/4418 [00:01<04:33, 16.06it/s]


  1%|          | 24/4418 [00:01<04:32, 16.12it/s]


  1%|          | 26/4418 [00:01<04:33, 16.03it/s]


  1%|          | 28/4418 [00:01<04:42, 15.57it/s]


  1%|          | 30/4418 [00:01<04:36, 15.85it/s]


  1%|          | 32/4418 [00:02<04:36, 15.89it/s]


  1%|          | 34/4418 [00:02<04:36, 15.87it/s]


  1%|          | 36/4418 [00:02<04:35, 15.88it/s]


  1%|          | 38/44

  6%|▌         | 266/4418 [00:21<06:34, 10.52it/s]


  6%|▌         | 268/4418 [00:21<06:35, 10.49it/s]


  6%|▌         | 270/4418 [00:21<06:37, 10.44it/s]


  6%|▌         | 272/4418 [00:21<06:37, 10.44it/s]


  6%|▌         | 274/4418 [00:21<06:37, 10.44it/s]


  6%|▌         | 276/4418 [00:22<06:37, 10.41it/s]


  6%|▋         | 278/4418 [00:22<06:38, 10.40it/s]


  6%|▋         | 280/4418 [00:22<06:38, 10.37it/s]


  6%|▋         | 282/4418 [00:22<06:38, 10.37it/s]


  6%|▋         | 284/4418 [00:22<06:41, 10.31it/s]


  6%|▋         | 286/4418 [00:23<06:44, 10.22it/s]


  7%|▋         | 288/4418 [00:23<06:46, 10.17it/s]


  7%|▋         | 290/4418 [00:23<06:48, 10.11it/s]


  7%|▋         | 292/4418 [00:23<06:48, 10.11it/s]


  7%|▋         | 294/4418 [00:23<06:49, 10.07it/s]


  7%|▋         | 296/4418 [00:24<06:51, 10.02it/s]


  7%|▋         | 298/4418 [00:24<06:52,  9.98it/s]


  7%|▋         | 299/4418 [00:24<06:57,  9.86it/s]


  7%|▋         | 300/4418 [00:24<06:59,  9.81i

  9%|▉         | 414/4418 [00:37<08:15,  8.08it/s]


  9%|▉         | 415/4418 [00:37<08:42,  7.65it/s]


  9%|▉         | 416/4418 [00:37<08:24,  7.93it/s]


  9%|▉         | 417/4418 [00:37<08:11,  8.14it/s]


  9%|▉         | 418/4418 [00:37<08:04,  8.25it/s]


  9%|▉         | 419/4418 [00:37<08:02,  8.29it/s]


 10%|▉         | 420/4418 [00:37<07:58,  8.35it/s]


 10%|▉         | 421/4418 [00:38<08:02,  8.28it/s]


 10%|▉         | 422/4418 [00:38<08:26,  7.89it/s]


 10%|▉         | 423/4418 [00:38<08:16,  8.05it/s]


 10%|▉         | 424/4418 [00:38<08:06,  8.22it/s]


 10%|▉         | 425/4418 [00:38<08:00,  8.31it/s]


 10%|▉         | 426/4418 [00:38<07:57,  8.35it/s]


 10%|▉         | 427/4418 [00:38<07:58,  8.35it/s]


 10%|▉         | 428/4418 [00:38<07:57,  8.35it/s]


 10%|▉         | 429/4418 [00:39<08:15,  8.05it/s]


 10%|▉         | 430/4418 [00:39<08:25,  7.89it/s]


 10%|▉         | 431/4418 [00:39<08:35,  7.74it/s]


 10%|▉         | 432/4418 [00:39<08:42,  7.63i

 12%|█▏        | 546/4418 [00:54<09:11,  7.03it/s]


 12%|█▏        | 547/4418 [00:54<09:05,  7.09it/s]


 12%|█▏        | 548/4418 [00:54<09:08,  7.05it/s]


 12%|█▏        | 549/4418 [00:54<08:58,  7.18it/s]


 12%|█▏        | 550/4418 [00:54<08:53,  7.25it/s]


 12%|█▏        | 551/4418 [00:54<08:47,  7.33it/s]


 12%|█▏        | 552/4418 [00:54<09:00,  7.15it/s]


 13%|█▎        | 553/4418 [00:55<09:14,  6.97it/s]


 13%|█▎        | 554/4418 [00:55<09:29,  6.79it/s]


 13%|█▎        | 555/4418 [00:55<09:40,  6.65it/s]


 13%|█▎        | 556/4418 [00:55<09:25,  6.83it/s]


 13%|█▎        | 557/4418 [00:55<09:15,  6.95it/s]


 13%|█▎        | 558/4418 [00:55<09:23,  6.85it/s]


 13%|█▎        | 559/4418 [00:56<09:29,  6.77it/s]


 13%|█▎        | 560/4418 [00:56<09:14,  6.96it/s]


 13%|█▎        | 561/4418 [00:56<09:21,  6.86it/s]


 13%|█▎        | 562/4418 [00:56<09:28,  6.79it/s]


 13%|█▎        | 563/4418 [00:56<09:14,  6.95it/s]


 13%|█▎        | 564/4418 [00:56<09:06,  7.05i

 15%|█▌        | 678/4418 [01:13<09:29,  6.56it/s]


 15%|█▌        | 679/4418 [01:13<09:25,  6.61it/s]


 15%|█▌        | 680/4418 [01:13<09:28,  6.57it/s]


 15%|█▌        | 681/4418 [01:14<09:32,  6.52it/s]


 15%|█▌        | 682/4418 [01:14<09:29,  6.57it/s]


 15%|█▌        | 683/4418 [01:14<09:26,  6.59it/s]


 15%|█▌        | 684/4418 [01:14<09:25,  6.61it/s]


 16%|█▌        | 685/4418 [01:14<09:29,  6.56it/s]


 16%|█▌        | 686/4418 [01:14<09:33,  6.51it/s]


 16%|█▌        | 687/4418 [01:14<09:34,  6.49it/s]


 16%|█▌        | 688/4418 [01:15<09:35,  6.48it/s]


 16%|█▌        | 689/4418 [01:15<09:51,  6.31it/s]


 16%|█▌        | 690/4418 [01:15<09:43,  6.39it/s]


 16%|█▌        | 691/4418 [01:15<09:36,  6.47it/s]


 16%|█▌        | 692/4418 [01:15<09:32,  6.51it/s]


 16%|█▌        | 693/4418 [01:15<09:30,  6.53it/s]


 16%|█▌        | 694/4418 [01:16<09:34,  6.48it/s]


 16%|█▌        | 695/4418 [01:16<09:38,  6.44it/s]


 16%|█▌        | 696/4418 [01:16<09:38,  6.43i

...

 87%|████████▋ | 3824/4418 [19:19<05:21,  1.85it/s]


 87%|████████▋ | 3825/4418 [19:20<05:26,  1.82it/s]


 87%|████████▋ | 3826/4418 [19:21<05:29,  1.79it/s]


 87%|████████▋ | 3827/4418 [19:21<05:27,  1.81it/s]


 87%|████████▋ | 3828/4418 [19:22<05:25,  1.81it/s]


 87%|████████▋ | 3829/4418 [19:22<05:24,  1.82it/s]


 87%|████████▋ | 3830/4418 [19:23<05:21,  1.83it/s]


 87%|████████▋ | 3831/4418 [19:23<05:19,  1.84it/s]


 87%|████████▋ | 3832/4418 [19:24<05:19,  1.83it/s]


 87%|████████▋ | 3833/4418 [19:24<05:16,  1.85it/s]


 87%|████████▋ | 3834/4418 [19:25<05:16,  1.85it/s]


 87%|████████▋ | 3835/4418 [19:25<05:16,  1.84it/s]


 87%|████████▋ | 3836/4418 [19:26<05:15,  1.84it/s]


 87%|████████▋ | 3837/4418 [19:27<05:15,  1.84it/s]


 87%|████████▋ | 3838/4418 [19:27<05:15,  1.84it/s]


 87%|████████▋ | 3839/4418 [19:28<05:14,  1.84it/s]


 87%|████████▋ | 3840/4418 [19:28<05:13,  1.84it/s]


 87%|████████▋ | 3841/4418 [19:29<05:12,  1.84it/s]


 87%|████████▋ | 3842/4418 [

 90%|████████▉ | 3959/4418 [20:31<04:15,  1.80it/s]


 90%|████████▉ | 3960/4418 [20:32<04:15,  1.79it/s]


 90%|████████▉ | 3961/4418 [20:32<04:15,  1.79it/s]


 90%|████████▉ | 3962/4418 [20:33<04:14,  1.79it/s]


 90%|████████▉ | 3963/4418 [20:33<04:14,  1.79it/s]


 90%|████████▉ | 3964/4418 [20:34<04:13,  1.79it/s]


 90%|████████▉ | 3965/4418 [20:34<04:12,  1.79it/s]


 90%|████████▉ | 3966/4418 [20:35<04:12,  1.79it/s]


 90%|████████▉ | 3967/4418 [20:36<04:12,  1.79it/s]


 90%|████████▉ | 3968/4418 [20:36<04:12,  1.78it/s]


 90%|████████▉ | 3969/4418 [20:37<04:11,  1.78it/s]


 90%|████████▉ | 3970/4418 [20:37<04:11,  1.78it/s]


 90%|████████▉ | 3971/4418 [20:38<04:10,  1.78it/s]


 90%|████████▉ | 3972/4418 [20:38<04:09,  1.79it/s]


 90%|████████▉ | 3973/4418 [20:39<04:08,  1.79it/s]


 90%|████████▉ | 3974/4418 [20:39<04:08,  1.79it/s]


 90%|████████▉ | 3975/4418 [20:40<04:08,  1.78it/s]


 90%|████████▉ | 3976/4418 [20:41<04:09,  1.77it/s]


 90%|█████████ | 3977/4418 [

 93%|█████████▎| 4100/4418 [21:45<02:43,  1.94it/s]


 93%|█████████▎| 4101/4418 [21:45<02:49,  1.87it/s]


 93%|█████████▎| 4102/4418 [21:46<02:52,  1.84it/s]


 93%|█████████▎| 4103/4418 [21:46<02:53,  1.81it/s]


 93%|█████████▎| 4104/4418 [21:47<02:55,  1.79it/s]


 93%|█████████▎| 4105/4418 [21:48<02:55,  1.78it/s]


 93%|█████████▎| 4106/4418 [21:48<02:57,  1.76it/s]


 93%|█████████▎| 4107/4418 [21:49<02:57,  1.75it/s]


 93%|█████████▎| 4108/4418 [21:49<02:58,  1.74it/s]


 93%|█████████▎| 4109/4418 [21:50<02:57,  1.74it/s]


 93%|█████████▎| 4110/4418 [21:51<02:56,  1.74it/s]


 93%|█████████▎| 4111/4418 [21:51<02:56,  1.74it/s]


 93%|█████████▎| 4112/4418 [21:52<02:55,  1.74it/s]


 93%|█████████▎| 4113/4418 [21:52<02:54,  1.74it/s]


 93%|█████████▎| 4114/4418 [21:53<02:54,  1.74it/s]


 93%|█████████▎| 4115/4418 [21:53<02:54,  1.73it/s]


 93%|█████████▎| 4116/4418 [21:54<02:54,  1.73it/s]


 93%|█████████▎| 4117/4418 [21:55<02:53,  1.73it/s]


 93%|█████████▎| 4118/4418 [

 96%|█████████▌| 4231/4418 [23:01<01:39,  1.87it/s]


 96%|█████████▌| 4232/4418 [23:01<01:42,  1.81it/s]


 96%|█████████▌| 4233/4418 [23:02<01:44,  1.78it/s]


 96%|█████████▌| 4234/4418 [23:02<01:44,  1.75it/s]


 96%|█████████▌| 4235/4418 [23:03<01:45,  1.73it/s]


 96%|█████████▌| 4236/4418 [23:04<01:46,  1.72it/s]


 96%|█████████▌| 4237/4418 [23:04<01:46,  1.71it/s]


 96%|█████████▌| 4238/4418 [23:05<01:46,  1.70it/s]


 96%|█████████▌| 4239/4418 [23:05<01:45,  1.69it/s]


 96%|█████████▌| 4240/4418 [23:06<01:45,  1.70it/s]


 96%|█████████▌| 4241/4418 [23:07<01:44,  1.69it/s]


 96%|█████████▌| 4244/4418 [23:07<01:22,  2.11it/s]


 96%|█████████▌| 4245/4418 [23:08<01:28,  1.96it/s]


 96%|█████████▌| 4246/4418 [23:08<01:32,  1.87it/s]


 96%|█████████▌| 4247/4418 [23:09<01:34,  1.80it/s]


 96%|█████████▌| 4248/4418 [23:09<01:36,  1.77it/s]


 96%|█████████▌| 4249/4418 [23:10<01:36,  1.74it/s]


 96%|█████████▌| 4250/4418 [23:11<01:37,  1.72it/s]


 96%|█████████▌| 4251/4418 [

 99%|█████████▉| 4364/4418 [24:19<00:33,  1.63it/s]


 99%|█████████▉| 4365/4418 [24:19<00:32,  1.63it/s]


 99%|█████████▉| 4366/4418 [24:20<00:32,  1.61it/s]


 99%|█████████▉| 4367/4418 [24:21<00:31,  1.61it/s]


 99%|█████████▉| 4368/4418 [24:21<00:30,  1.62it/s]


 99%|█████████▉| 4369/4418 [24:22<00:30,  1.62it/s]


 99%|█████████▉| 4370/4418 [24:22<00:29,  1.62it/s]


 99%|█████████▉| 4371/4418 [24:23<00:28,  1.63it/s]


 99%|█████████▉| 4372/4418 [24:24<00:28,  1.63it/s]


 99%|█████████▉| 4373/4418 [24:24<00:27,  1.64it/s]


 99%|█████████▉| 4374/4418 [24:25<00:26,  1.63it/s]


 99%|█████████▉| 4375/4418 [24:25<00:26,  1.63it/s]


 99%|█████████▉| 4376/4418 [24:26<00:25,  1.63it/s]


 99%|█████████▉| 4377/4418 [24:27<00:25,  1.63it/s]


 99%|█████████▉| 4379/4418 [24:27<00:20,  1.91it/s]


 99%|█████████▉| 4380/4418 [24:28<00:20,  1.81it/s]


 99%|█████████▉| 4381/4418 [24:29<00:21,  1.75it/s]


 99%|█████████▉| 4382/4418 [24:29<00:20,  1.72it/s]


 99%|█████████▉| 4383/4418 [

In [114]:
import folium
from folium import plugins

map1 = folium.Map(
    location=[48.150305,11.580054],
    tiles='stamentoner',
    zoom_start=12,
)

datas = []
for idx, coord_list in enumerate(coord_dataframe_list):
    if not coord_list.empty:
        coord_list = coord_list.assign(norm_total=(coord_list.total/coord_list.total.max()))
        coord_list.reset_index(inplace=True)
        datas.append(coord_list[['timestamp', 'latitude', 'longitude', 'total']].values.tolist())

# map1.add_child(hm)
# List comprehension to make out list of lists
# heat_df = coord_dataframe_list[0]

sl = []
for idx, elem in enumerate(datas[0]):
    temp = []
    for i in range(len(datas)):
        try:
            temp.append([datas[i][idx][1], datas[i][idx][2], datas[i][idx][3]])
        except IndexError:
            pass
    sl.append(temp)

# hm = plugins.HeatMap(data, max_val=coord_dataframe_list[0]['total'].max())
# heat_data = [[[row['latitude'],row['longitude']] for index, row in heat_df[heat_df['norm_total'] == i].iterrows()] for i in range(0,len(heat_df))]
hmt = plugins.HeatMapWithTime(sl,auto_play=True,use_local_extrema=True, max_opacity=0.8,index=coord_dataframe_list[0].reset_index().timestamp.tolist())
hmt.add_to(map1)

# coord_dataframe_list[0].apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)

#map1
map1.save('map10.html')

In [103]:
coord_dataframe_list[0].reset_index().timestamp.to_list()

[1557011700,
 1557012600,
 1557013500,
 1557014400,
 1557015300,
 1557016200,
 1557017100,
 1557018000,
 1557018900,
 1557019800,
 1557020700,
 1557021600,
 1557022500,
 1557023400,
 1557024300,
 1557025200,
 1557026100,
 1557027000,
 1557027900,
 1557028800,
 1557029700,
 1557030600,
 1557031500,
 1557032400,
 1557033300,
 1557034200,
 1557035100,
 1557036000,
 1557036900,
 1557037800,
 1557038700,
 1557039600,
 1557040500,
 1557041400,
 1557042300,
 1557043200,
 1557044100,
 1557045000,
 1557045900,
 1557046800,
 1557047700,
 1557048600,
 1557049500,
 1557050400,
 1557051300,
 1557052200,
 1557053100,
 1557054000,
 1557054900,
 1557055800,
 1557056700,
 1557057600,
 1557058500,
 1557059400,
 1557060300,
 1557061200,
 1557062100,
 1557063000,
 1557063900,
 1557064800,
 1557065700,
 1557066600,
 1557067500,
 1557068400,
 1557069300,
 1557070200,
 1557071100,
 1557072000,
 1557072900,
 1557073800,
 1557074700,
 1557075600,
 1557076500,
 1557077400,
 1557078300,
 1557079200,
 1557080100,

In [115]:
for g,s in filtered_desc.groupby('Standort'):
    break

In [117]:
s

,ap,latitude,longitude,Standort
4310,apa01-0lj,48.15788,11.58357,"ABZ, Ausbildungszentrum für Pastoralreferenten..."
4311,apa01-klj,48.15788,11.58357,"ABZ, Ausbildungszentrum für Pastoralreferenten..."


In [113]:
s[0]

KeyError: 'Column not found: 0'

In [119]:
s = [[1,2,3], [1,4,5]]

In [123]:
[item[0] for item in s]

[1, 1]